# 11 Pontos de Precisão e Recall

In [62]:
import pandas as pd
import numpy as np

In [97]:
result = pd.read_csv('../resultados_sem_stemmer/RESULTADOS-NOSTEMMER.csv', sep=';')
result['Position'] = result['Results'].str.split(', ', expand=True)[0].str[1:]
result['DocNumber'] = result['Results'].str.split(', ', expand=True)[1]
result['Relevance'] = result['Results'].str.split(', ', expand=True)[2].str[:-1]
result.drop(columns=['Results'], inplace=True)
result

,QueryNumber,Position,DocNumber,Relevance
0,1,1.0,40,0.07736125211124294
1,1,2.0,971,0.050234117172333045
2,1,3.0,302,0.047962021158067886
3,1,4.0,1188,0.046768682198753596
4,1,5.0,392,0.046238516757099904
...,...,...,...,...
120280,100,1211.0,1230,0.0008328031501789552
120281,100,1212.0,974,0.0005873420398677124
120282,100,1213.0,948,0.0005235496427597196
120283,100,1214.0,940,0.0004357997398344579


In [99]:
result_stemmer = pd.read_csv('../resultados_com_stemmer/RESULTADOS-STEMMER.csv', sep=';')
result_stemmer['Position'] = result_stemmer['Results'].str.split(', ', expand=True)[0].str[1:]
result_stemmer['DocNumber'] = result_stemmer['Results'].str.split(', ', expand=True)[1]
result_stemmer['Relevance'] = result_stemmer['Results'].str.split(', ', expand=True)[2].str[:-1]
result_stemmer.drop(columns=['Results'], inplace=True)
result_stemmer

,QueryNumber,Position,DocNumber,Relevance
0,1,1.0,40,0.07565648960798238
1,1,2.0,971,0.048856105525199256
2,1,3.0,392,0.04711830903336669
3,1,4.0,1026,0.04371218236054627
4,1,5.0,302,0.043700336931355756
...,...,...,...,...
120280,100,1211.0,547,0.0008780403865268196
120281,100,1212.0,1230,0.0008363791324622214
120282,100,1213.0,974,0.0005898640340848728
120283,100,1214.0,948,0.0005257977181260492


In [34]:
esperados = pd.read_csv('../resultados_sem_stemmer/esperados.csv', sep=';')
esperados

,QueryNumber,DocNumber,DocVotes
0,1,139,7
1,1,151,6
2,1,166,1
3,1,311,1
4,1,370,2
...,...,...,...
4815,100,1000,6
4816,100,1017,7
4817,100,1033,1
4818,100,1097,2


In [104]:
a = [1,2,3]
b = [3,4,5]

a%b

TypeError: unsupported operand type(s) for &: 'list' and 'list'

In [108]:
queries = esperados['QueryNumber'].unique()
precision = {}
recall = {}

for query in queries:
    precision[query] = []
    recall[query] = []
    relevant_documents = esperados[esperados['QueryNumber'] == query]['DocNumber']
    results = result[result['QueryNumber'] == query]['DocNumber']
    for i in range(11):
        results_until_i = results[:(i+1)*len(results)//11]
        same_elements = np.intersect1d(relevant_documents, results_until_i)
    
        precision[query].append(len(same_elements)/len(results_until_i))
        recall[query].append(len(same_elements)/len(relevant_documents))  

    

0        40
1       971
2       302
3      1188
4       392
       ... 
105     497
106     499
107    1215
108     969
109     772
Name: DocNumber, Length: 110, dtype: object
0        40
1       971
2       302
3      1188
4       392
       ... 
215    1014
216      39
217     988
218     316
219    1180
Name: DocNumber, Length: 220, dtype: object
0        40
1       971
2       302
3      1188
4       392
       ... 
326     923
327     718
328     312
329    1051
330     815
Name: DocNumber, Length: 331, dtype: object
0        40
1       971
2       302
3      1188
4       392
       ... 
436     778
437     953
438     775
439      15
440     361
Name: DocNumber, Length: 441, dtype: object
0        40
1       971
2       302
3      1188
4       392
       ... 
547     797
548      46
549     702
550     378
551     414
Name: DocNumber, Length: 552, dtype: object
0        40
1       971
2       302
3      1188
4       392
       ... 
657     432
658     367
659     859
660     289


4809      59
4810     183
4811     370
4812     579
4813     803
4814     833
4815    1000
4816    1017
4817    1033
4818    1097
4819    1232
Name: DocNumber, dtype: int64